### Read training, dev and unlabeled test data

The following provides a starting code (Python 3) of how to read the labeled training and dev cipher text, and unlabeled test cipher text, into lists.

In [1]:
train, dev, test = [], [], []

In [2]:
for x in open('./train_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    train.append(x)
print (len(train))
print (train[:3])

16220
[[0, 'lkêcê yoúc cêêö y#êjl lw mówám Újám j Úêê# ütlk Úol lkêú z#ê ctöé8ú ówl xoóóú éê#xw#öê#c .'], [0, '6êcétlê jolêot8 zc éê#xw#öjóáê , tl zc j #jlkê# 8tcl8êcc jöÚ8ê 6wüó lkê öt668ê wx lkê #wj6 , ükê#ê lkê lkêöjltá t#wótêc j#ê lww wÚ2twoc jó6 lkê cê+oj8 éw8tltác lww cöoy .'], [0, 'tx lktc kw8t6jú öw2tê tc coééwcê6 lw Úê j ytxl , cwöêÚw6ú oóü#jééê6 tl êj#8ú , lwwm wol j88 lkê yww6 cloxx , jó6 8êxl Úêktó6 lkê á#jé ( 8tlê#j88ú ) .']]


In [3]:
for x in open('./dev_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    dev.append(x)
print (len(dev))
print (dev[:3])

2027
[[1, 'ów8jó Ú#j2ê8ú l#êj6c ükê#ê xêü jöê#tájó xt8öc 6j#ê lw 6ê82ê 77 tólw lkê üw#86 wx jöÚt2j8êóáê jó6 jöÚtyotlú <<<'], [0, 'ê2êó öo#ékú zc ê+éê#l áwötá ltötóy jó6 xjöê6 ákj#tcöj áj ózl #êcáoê lktc êxxw#l .'], [1, 'üt88 jcco#ê68ú #jóm jc wóê wx lkê á8ê2ê#êcl , öwcl 6êáêélt2ê8ú jöoctóy áwöê6têc wx lkê úêj# .']]


#### Different from 'train' and 'dev' that are both list of tuples, 'test' will be just a list.

In [4]:
for x in open('./test_enc_unlabeled.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r')
    test.append(x)
print (len(test))
print (test[:3])

2028
['j 6t6jáltá jó6 6o88 6wáoöêólj#ú y8w#txútóy cwxlüj#ê jój#ákú .', 'ówlktóy cltámc , #êj88ú , ê+áêél j 8tóyê#tóy á#êêétóêcc wóê xêê8c x#wö Úêtóy 6#jyyê6 lk#woyk j cj6 , cw#6t6 oót2ê#cê wx yoóc , 6#oyc , j2j#táê jó6 6jöjyê6 6#êjöc .', 'öo#ékú jó6 üt8cwó jáloj88ú öjmê j é#êllú yww6 lêjö <<< Úol lkê é#wvêál co##woó6tóy lkêö tc 6tcl#êcctóy8ú #wlê .']


#### You can split every sentence into lists of words by white spaces.

In [5]:
train_split = [[x[0], x[1].split(' ')] for x in train]
dev_split = [[x[0], x[1].split(' ')] for x in dev]
test_split = [[x.split(' ')] for x in test]

### Main Code Body

You may choose to experiment with different methods using your program. However, you need to embed the training and inference processes at here. We will use your prediction on the unlabeled test data to grade, while checking this part to understand how your method has produced the predictions.

In [6]:
train_sentences = [el[1] for el in train]
y_train = [el[0] for el in train]
print(len(train_sentences))
train_sentences[:2]
y_train[:4]

16220


[0, 0, 0, 1]

In [7]:
dev_sentences = [el[1] for el in dev]
y_dev = [el[0] for el in dev]
dev_sentences[:3]
y_dev[:4]

[1, 0, 1, 1]

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=10000, min_df=3, max_df=0.8)
X_train = vectorizer.fit_transform(train_sentences).toarray()
print(len(vectorizer.vocabulary_))

8198


In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X_train = tfidfconverter.fit_transform(X_train).toarray()
print(len(X_train[0]))

8198


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(vocabulary=vectorizer.vocabulary_, min_df=3, max_df=0.8)
X_dev = tfidfconverter.fit_transform(dev_sentences).toarray()
print(len(X_dev[0]))

8198


In [11]:
from sklearn.ensemble import RandomForestClassifier
randomForest = RandomForestClassifier(n_estimators=10, random_state=0)
randomForest.fit(X_train, y_train) 

RandomForestClassifier(n_estimators=10, random_state=0)

In [12]:
y_pred_rf = randomForest.predict(X_dev)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_dev,y_pred_rf))
print(classification_report(y_dev,y_pred_rf))
print(accuracy_score(y_dev, y_pred_rf))

[[824 128]
 [239 836]]
              precision    recall  f1-score   support

           0       0.78      0.87      0.82       952
           1       0.87      0.78      0.82      1075

    accuracy                           0.82      2027
   macro avg       0.82      0.82      0.82      2027
weighted avg       0.82      0.82      0.82      2027

0.8189442525900346


In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
base_svc = SVC()
adaBoost = AdaBoostClassifier(base_estimator=base_svc,n_estimators=10,algorithm='SAMME')
adaBoost.fit(X_train, y_train)

In [15]:
y_pred_ab = adaBoost.predict(X_dev)

In [16]:
print(confusion_matrix(y_dev,y_pred_ab))
print(classification_report(y_dev,y_pred_ab))
print(accuracy_score(y_dev, y_pred_ab))

[[   0  952]
 [   0 1075]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       952
           1       0.53      1.00      0.69      1075

    accuracy                           0.53      2027
   macro avg       0.27      0.50      0.35      2027
weighted avg       0.28      0.53      0.37      2027

0.5303404045387272


C:\Users\suviy\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\suviy\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\suviy\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
parameters = {'kernel':['poly', 'rbf', 'sigmoid'],'C':[1,2,3], 'probability':[True,False]}
svc_hyper = SVC()
svc_hyper_tun = GridSearchCV(svc_hyper,parameters)
svc_hyper_tun.fit(X_train,y_train)

In [ ]:
sorted(svc_hyper_tun.cv_results_.keys())

### Output Prediction Result File

You will need to submit a prediction result file. It should have 2028 lines, every line should be either 0 or 1, which is your model's prediction on the respective test set instance.

In [ ]:
# suppose you had your model's predictions on the 2028 test cases read from test_enc_unlabeled.tsv, and 
#those results are in the list called 'results'
assert (len(results) == 2028)

In [ ]:
# make sure the results are not float numbers, but intergers 0 and 1
results = [int(x) for x in results]

In [ ]:
# write your prediction results to 'upload_predictions.txt' and upload that later
with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
    for x in results:
        fp.write(str(x) + '\n')